This is the section for the group project proposal for group 9!

Here is the link to the data set we will be using, Online News Populartiy from the UCI Machine Learning Resprotory.
Subsiquent sources can be found in the link on the UCI web page.
Link: https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity